In [1]:
import pandas as pd
import polars as pl
import sqlalchemy as sa
import urllib
from datetime import datetime, timedelta
import math
import numpy as np
import datetime as dt
import glob
import numpy as np

# AKC LOGIN:
## User: operator_ict
## Passwork: Ict@123456

In [6]:
# Note
# Ngày 6/3 đổi tỷ lệ điểm từ 40,000VND/Điểm -> 4,000VND/Điểm
# Burning thì hoàn toàn là data mớig
# Partner Refund thì tự net-off với phiếu song son
# Invoice OK đã lấy theo điểm mới
#

In [2]:
def getEngine(engine_name):
    connection = f"mssql+pyodbc:///?odbc_connect={urllib.parse.quote_plus(engine_name)}"
    engine = sa.create_engine(connection, fast_executemany=True)
    return engine
    
### Connect to DataMart Server
mart_server = '118.69.201.34'
mart_database = 'FLC_SHOP_SALES_DATAWAREHOUSE'
mart_username = 'ecom_user'
mart_password = 'Ec0m@12345'
SaleMart = ('DRIVER={ODBC Driver 17 for SQL Server}'
            ';SERVER=' + mart_server +
            ';DATABASE=' + mart_database +
            ';UID=' + mart_username +
            ';PWD=' + mart_password)

### Connect to Azure server
azr_server = "ssd-synapse-prod.sql.azuresynapse.net"
azr_database = 'frtssdsqlpoolprod'
azr_username = 'frtcid'
azr_password = 'Frt2022Cid@#123'
        
connection_url = sa.engine.URL.create(
    "mssql+pyodbc",
    username=azr_username,
    password=azr_password,
    host=azr_server,
    database=azr_database,
    query={
        "driver": "ODBC Driver 17 for SQL Server",
        "autocommit": "True",
    },
)

azr_engine = sa.create_engine(connection_url)

In [3]:
source_ = 'Data_20230629_20230629'
df = pd.read_excel(r"D:/IMPORT_AKC/2023-06/" + source_ + ".xlsx")

df['Ngày tạo']          = pd.to_datetime(df['Ngày tạo'])
df['Ngày giao dịch']    = pd.to_datetime(df['Ngày giao dịch'])

col_ = ['Mã thành viên', 'Ngày tạo', 'Ngày giao dịch', 'Lý do', 'Mã giao dịch',
       'Số tiền mua', 'Điểm xếp hạng quy đổi', 'Điểm phát sinh',
       'Điểm đổi quà đã sử dụng']
df = df[col_]
df['TIME_INSERT']   = datetime.now()
df['Source']        = source_
min_date_filter_    = df['Ngày tạo'].min() - timedelta(days=5)
df.sample(3)

,Mã thành viên,Ngày tạo,Ngày giao dịch,Lý do,Mã giao dịch,Số tiền mua,Điểm xếp hạng quy đổi,Điểm phát sinh,Điểm đổi quà đã sử dụng,TIME_INSERT,Source
2204,2248aa17-02d8-487c-b66c-5fef8b2dceb2,2023-06-29,2023-06-29,Invoice,45294445,2990000.0,747,747,NaN,2023-06-30 09:17:44.730910,Data_20230629_20230629
5758,b7ec2041-8c87-4cf4-b15e-f0656301473f,2023-06-29,2023-06-29,Invoice,45289920,18690000.0,4672,4672,NaN,2023-06-30 09:17:44.730910,Data_20230629_20230629
1021,109df0fb-0675-48e3-82f3-ec3ffa34a347,2023-06-29,2023-06-29,Invoice,45296032,9000.0,2,2,NaN,2023-06-30 09:17:44.730910,Data_20230629_20230629


In [4]:
## Lấy mã giao dịch dò trong Server Azure để tìm ra mã khách hàng

list_inv = df[df['Lý do']=='Invoice']['Mã giao dịch'].drop_duplicates()
num_part = math.ceil(len(list_inv) /15_000)

list_cus_inv_out = pd.DataFrame()
print(f"Number of records = {len(list_inv):,.0f}")
for i in range(1,num_part+1):
    list_ = list_inv[(i-1)*15_000 : min(i * 15_000, len(list_inv))]
    num_record = min(i * 15_000, len(list_inv))
    print(f"|_{i}_{num_record:,.0f}_{num_record/len(list_inv):,.0%}")
    list_ = ','.join(list_)
    query_read = f"""SELECT DISTINCT a.CUSTOMER_KEY, a.INVOICE_HEADER 
                FROM VF_CID_SALES_TRANSACTION_ICT a
                LEFT JOIN (SELECT value as 'INVOICE_HEADER' FROM STRING_SPLIT( '{list_}' , ',')) b
                    ON a.INVOICE_HEADER = b.INVOICE_HEADER
                WHERE b.INVOICE_HEADER IS NOT NULL"""       
    list_cus_inv = pd.read_sql(query_read, azr_engine)
    list_cus_inv_out = pd.concat([list_cus_inv_out, list_cus_inv])
list_cus_inv_out['INVOICE_HEADER']=list_cus_inv_out['INVOICE_HEADER'].apply(str)

Number of records = 8,303
|_1_8,303_100%


E:\Anaconda\lib\site-packages\pandas\io\sql.py:1760: SAWarning: Could not fetch transaction isolation level, tried views: ('sys.dm_exec_sessions', 'sys.dm_pdw_nodes_exec_sessions'); final error was: ('42000', '[42000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]User does not have permission to perform this action. (6004) (SQLExecDirectW)')
  insp = sqlalchemy_inspect(self.connectable)


In [5]:
list_cus_inv_out.head()

,CUSTOMER_KEY,INVOICE_HEADER
0,1771569,45297037
1,2395395,45290649
2,2475783,45289227
3,2584624,45296343
4,2830357,45289449


In [6]:
## Check danh sách khách hàng để tạo DIM 

check_unique_cus = df[df['Lý do']=='Invoice']\
    .merge(
        list_cus_inv_out, 
        left_on= 'Mã giao dịch', 
        right_on = 'INVOICE_HEADER', 
        how = 'left'
        )
df_mapping_cus      = check_unique_cus[check_unique_cus['CUSTOMER_KEY'].notna()][['CUSTOMER_KEY','Mã thành viên']].drop_duplicates()
check_unique_cus    = df_mapping_cus.groupby('Mã thành viên', as_index=False).agg(NumCus = ('CUSTOMER_KEY','nunique')).sort_values(by='NumCus',ascending=False)
len_not_unique      = len(check_unique_cus[check_unique_cus['NumCus']>1])

if len_not_unique != 0:
    print("Error, has not unique code", len_not_unique)
else:
    print("Has no duplicate, can use")
    print(f"Len of df: {len(df_mapping_cus):,.0f}")

## Check với DIM đã có trên server
## Chỉ import những mã khách hàng chưa có

query_read_cus = """SELECT [Mã thành viên] FROM D_ICT_AKACHAIN_LOYALTY_CUS_MAPPING """
list_cus_old = pd.read_sql(query_read_cus, getEngine(SaleMart))
df_mapping_cus = df_mapping_cus[~df_mapping_cus['Mã thành viên'].isin(list_cus_old['Mã thành viên'])]
print(f"Length of import = {len(df_mapping_cus):,.0f}")


## Get basic Info of customer base on new customer
if len(df_mapping_cus) > 0:
    num_part = math.ceil(len(df_mapping_cus) /15_000)
    list_cus_get_info = df_mapping_cus['CUSTOMER_KEY'].astype(str)
    list_cus_info_out = pd.DataFrame()
    print(f"Number of records = {len(df_mapping_cus):,.0f}")
    for i in range(1, num_part+1):
        list_ = list_cus_get_info[(i-1)*15_000 : min(i * 15_000, len(list_cus_get_info))]
        print(f"{i} | {min(i * 15_000, len(list_cus_get_info)):,.0f} | {len(list_):,.0f}")
        list_ = ','.join(list_)
        query_read_info = f"""SELECT [CUSTOMER_KEY], [CUSTOMER_NAME], [CUSTOMER_ADDRESS], [CUSTOMER_PHONE] 
                        FROM [dbo].[VD_CID_CUSTOMER_ICT] WHERE CUSTOMER_KEY IN ( {list_} )"""
        list_cus_info = pd.read_sql(query_read_info, azr_engine)
        list_cus_info_out = pd.concat([list_cus_info_out, list_cus_info])

if len(df_mapping_cus) > 0 : 
    print("Start importing data customer")
    df_mapping_cus\
        .merge(list_cus_info_out, on='CUSTOMER_KEY', how='left')\
        .to_sql("D_ICT_AKACHAIN_LOYALTY_CUS_MAPPING", getEngine(SaleMart), if_exists='append', index=False)
    print("Import Done")
else: 
    print("Not import because duplicate")

Has no duplicate, can use
Len of df: 6,250
Length of import = 4,054
Number of records = 4,054
1 | 4,054 | 4,054
Start importing data customer
Import Done


In [7]:
query_read = f"""SELECT 
                DISTINCT [Ngày tạo], [Ngày giao dịch], [Mã giao dịch] 
                FROM [FLC_SHOP_SALES_DATAWAREHOUSE].[dbo].[F_ICT_AKACHAIN_LOYALTY] 
                WHERE [Ngày tạo] >=  '{str(min_date_filter_).split(' ',1)[0]}'
                """
history = pd.read_sql(query_read, getEngine(SaleMart))
history['Ngày tạo']         = pd.to_datetime(history['Ngày tạo'])
history['Ngày giao dịch']   = pd.to_datetime(history['Ngày giao dịch'])
history['Flg']              = 1
if len(history) > 0:
    history.sample(5)
else:
    print("Df null")

In [8]:
output = df.merge(
    history, 
    on = ['Ngày tạo', 'Ngày giao dịch','Mã giao dịch'], 
    how = 'left')
print(f"Number of duplicate: {len(output[output['Flg']==1]):,.0f}")
print(f"Number of not duplicate: {len(output[output['Flg']!=1]):,.0f}")

output = output[output['Flg']!=1]
output = output.drop(columns='Flg')
if len(output)>0:
    display(output.sample(3))
else:
    display("df Null")
    display(output.sample(3))

Number of duplicate: 0
Number of not duplicate: 8,627


,Mã thành viên,Ngày tạo,Ngày giao dịch,Lý do,Mã giao dịch,Số tiền mua,Điểm xếp hạng quy đổi,Điểm phát sinh,Điểm đổi quà đã sử dụng,TIME_INSERT,Source
1269,e99db969-d3de-4ba9-a1d5-e4e5f1e0879f,2023-06-29,2023-06-29,Invoice,45295716,287000.0,71,71,NaN,2023-06-30 09:17:44.730910,Data_20230629_20230629
8411,01e4c634-e960-4325-b1db-b0b2fb4b2f57,2023-06-29,2023-06-29,Invoice,45286255,135000.0,33,33,NaN,2023-06-30 09:17:44.730910,Data_20230629_20230629
7009,1600a3fd-7d27-4ce8-9c73-20439c9d5f03,2023-06-29,2023-06-29,Invoice,45288437,2390000.0,597,597,NaN,2023-06-30 09:17:44.730910,Data_20230629_20230629


In [9]:
output.to_sql(
    "F_ICT_AKACHAIN_LOYALTY", 
    getEngine(SaleMart), 
    if_exists='append', 
    index=False
    )

-1

In [10]:
## 15/03/2023 import thêm mapping return lên 
## Phải chạy code này mỗi lần import

query_read = f"""
            SELECT  
                DISTINCT a.INVOICE_HEADER, a.MEMO_HEADER
                , CAST(a.SALES_ORDER_DATE AS DATE) AS ReturnDate
                , CAST(a.TRANSACTION_DATE AS DATE) AS ReturnTransDate
                , CAST(b.SALES_ORDER_DATE AS DATE) AS SalesDate
                , CAST(b.TRANSACTION_DATE AS DATE) AS SalesTransDate
            FROM [VF_CID_SALES_TRANSACTION_ICT] a
            LEFT JOIN [VF_CID_SALES_TRANSACTION_ICT]  b
                ON a.INVOICE_HEADER = b.INVOICE_HEADER AND b.TRANSACTION_TYPE = 'Sales Invoice'
            WHERE 
                a.DATE_KEY > 20220701
                AND a.MEMO_HEADER IS NOT NULL 
                AND a.MEMO_HEADER <> 0
            """
df_mapping = pd.read_sql(query_read, azr_engine)
total_records_import = len(df_mapping)
print(f"Total records to import {total_records_import:,.0f}")
df_mapping.to_sql("F_ICT_AKACHAIN_MAPPING_RETURN", getEngine(SaleMart), if_exists='replace', index=False)

print("Done import")

Total records to import 32,787
Done import


In [11]:
# 04/05/2023 Import thêm mã GRPO để mapping
query_read = f"""
                SELECT 
                    GRPO_HEADER, CUSTOMER_CODE, SHOP_CODE, INVOICE_HEADER
                FROM VF_CID_GRPO_TRANSACTION_ICT
                WHERE SHOP_CODE IS NOT NULL
                """
df_grpo = pd.read_sql(query_read, azr_engine)
total_records_import = len(df_grpo)
print(f"Total records to import {total_records_import:,.0f}")
df_grpo.to_sql("F_ICT_AKACHAIN_MAPPING_GRPO", getEngine(SaleMart), if_exists='replace', index=False)
print("Done import")

Total records to import 8,576
Done import


## Hết phần import mỗi tuần


# Job audit dữ liệu định kỳ

## Step 1: Read dữ liệu

In [ ]:
import os
import pandas as pd
import polars as pl
# Read 
base_dir = r'C:\Users\Admin\OneDrive\FRT\6. Stored Data\AkachainMember\Project_AnualAudit'
dir_number = r'01_20230609'

all_path = []
for root, dirs, files in os.walk(os.path.join(base_dir,dir_number)):
    for file in files:
        all_path.append(os.path.join(base_dir, dir_number, file))

dfs = []
for _p in range(len(all_path)):    
        df = pl.read_excel(all_path[_p])
        df = df.with_columns(pl.lit(_p).alias('file_num'))
        dfs.append(df)
        print(f'\r Done get file number {_p}', end = '')

merged_df = pl.concat(dfs, )
merged_df = merged_df.with_columns(pl.col('Ngày tạo').str.strptime(pl.Date, fmt='%Y-%m-%d', strict=False).alias('Ngày tạo'))
merged_df = merged_df.with_columns(pl.col('Ngày giao dịch').str.strptime(pl.Date, fmt='%Y-%m-%d', strict=False).alias('Ngày giao dịch'))
merged_df = merged_df.with_column(pl.col("Ngày giao dịch").dt.strftime("%m").alias("month"))
merged_df.head() 


## Step 2: Check export duplicates

In [ ]:
# Check export có bị lỗi không
grouped_df = merged_df.groupby([pl.col('file_num'),pl.col('month')]).count().sort('file_num')
if len(grouped_df) == len(all_path):
    print(f"OK, not duplicated")
    print(grouped_df.head())
else:
    print(f"Error Duplicated")

## Step 3: Read history SQL

In [ ]:
min_date = '2023-01-01'
max_date = '2023-05-31'

query_read = f"""
    SELECT  [Mã thành viên]
      ,[Ngày tạo]
      ,[Ngày giao dịch]
      ,[Lý do]
      ,[Mã giao dịch]
      ,[Điểm phát sinh]
        FROM [FLC_SHOP_SALES_DATAWAREHOUSE].[dbo].[F_ICT_AKACHAIN_LOYALTY]
        WHERE [Ngày giao dịch] BETWEEN '{min_date}' AND '{max_date}'
"""
df_sql = pd.read_sql(query_read, getEngine(SaleMart))
df_sql = df_sql.astype({'Ngày tạo':'datetime64[ns]', 'Ngày giao dịch':'datetime64[ns]'})
df_sql.head()

In [ ]:
# So sánh df read_sql có duplicate không
length_df = len(df_sql)
length_df_dropdup = len(df_sql.drop_duplicates())
print(f"Length df: {length_df:,.0f}")
print(f"Length df drop duplicate: {length_df_dropdup:,.0f}")
print(f"Different = {length_df_dropdup - length_df :,.0f}")

In [ ]:
# Chuyển df Polars sang df_pandas
df_pd = merged_df.to_pandas()

In [ ]:
col_to_merge = ['Mã thành viên', 'Ngày tạo', 'Ngày giao dịch', 'Lý do', 'Mã giao dịch','Điểm phát sinh']

df_ss_pd = df_pd[col_to_merge]
df_ss = df_ss_pd.merge(df_sql, how = 'outer', indicator=True)

# Loại trừ mã khách hàng đặc biệt
df_special = df_ss[df_ss['Mã thành viên'] == '45605402-37aa-4fef-9cbb-6938256a372f']
df_ss = df_ss[df_ss['Mã thành viên'] != '45605402-37aa-4fef-9cbb-6938256a372f']

In [ ]:
# Stats:
df_ss.groupby('_merge').agg(_count =('_merge','count'))

In [ ]:
col_to_merge_2 = ['Mã thành viên', 'Ngày tạo', 'Ngày giao dịch', 'Lý do', 'Mã giao dịch']

df_ss_pd = df_pd[col_to_merge]
df_ss = df_ss_pd.merge(df_sql, how = 'outer', on = col_to_merge_2, indicator=True, suffixes=['_pd', '_sql'])

df_ss['diff'] = df_ss['Điểm phát sinh_pd'].fillna(0) - df_ss['Điểm phát sinh_sql'].fillna(0)
df_ss

In [ ]:
df_ss[df_ss['diff']!=0]

## Result to import

In [ ]:
# DF cuối cùng để import 

col_to_merge = ['Mã thành viên', 'Ngày tạo', 'Ngày giao dịch', 'Lý do', 'Mã giao dịch','Điểm phát sinh']

df_ss_pd = df_pd
df_ss = df_ss_pd.merge(df_sql, how = 'outer', indicator=True)

# Loại trừ mã khách hàng đặc biệt
df_special = df_ss[df_ss['Mã thành viên'] == '45605402-37aa-4fef-9cbb-6938256a372f']
df_ss = df_ss[df_ss['Mã thành viên'] != '45605402-37aa-4fef-9cbb-6938256a372f']
df_ss = df_ss[df_ss['_merge']!='both']
df_ss['TIME_INSERT'] = datetime.now()
df_ss['Source'] = 'Audit import ' + dir_number
df_ss.drop(columns=['_merge','file_num', 'month',],  inplace=True)

df_ss

In [ ]:
# Import final
df_ss.to_sql(
    "F_ICT_AKACHAIN_LOYALTY", 
    getEngine(SaleMart), 
    if_exists='append', 
    index=False
    )

In [ ]:
df_ss_pd

In [ ]:
len(df_ss.columns)

# Rewrite code import with identity column

In [ ]:
# Ngày 04/03/2023:
# Kỹ thuật identity column cho sql server để load dữ liệu nhanh hơn

# Read lại bảng cũ
df_import_cus = pd.read_sql("SELECT  * FROM [D_ICT_AKACHAIN_LOYALTY_CUS_MAPPING]", getEngine(SaleMart))
print('Done ')
# Import vào bảng đó
df_import_cus.to_sql('D_ICT_AKACHAIN_LOYALTY_CUS_MAPPING_v2', getEngine(SaleMart), index=False, if_exists='append')

# Dùng insert nhanh hơn đó
#

"DELETE FROM [D_ICT_AKACHAIN_LOYALTY_CUS_MAPPING_v2]"


"DBCC CHECKIDENT ('D_ICT_AKACHAIN_LOYALTY_CUS_MAPPING_v2', RESEED, 0)"

# Note check lại cách sử dụng bảng dim customer đã ổn chưa vì có nhiều trường hợp null, nên chăng đánh key từ bảng fact chứ ko phải DIM
# >> Không cần thiết phải đánh từ bảng DIM, mà là seeding sẵn vào bảng, nhưng khi có thông tin mới 
# >> 

# Flow : 
# 1.

Flow
1. Load data từ files hàng tuần
2. Check thông tin customer đó trên Azure (Transactions, key = BuyingInvoice_header)
3. Import thông tin customer vào Dim Customer những customer chưa được flag là completed và đánh identity nếu không có thì flag là completed
4. Import data AKC details vào DB

Note: Cẩn thận code làm sao nếu broken 1 tuần thì không phải cần chạy lại all data

Nhu cầu cuối cùng: Khi load bảng details lên Power BI thì chỉ cần load theo DIM là cột identity

## Generate danh sách file để chạy

In [ ]:
# Danh sách file để chạy
# Check lại trước khi chạy full log
import os, glob
import polars as pl
import pandas as pd
dir = r"C:/Users/Admin/OneDrive/FRT/6. Stored Data/AkachainMember/Data/"
file_list = glob.glob(os.path.join(dir, "*.xlsx"))
# file_list = [i for i in file_list if 'Data_' in i or 'Import' in i]
list_f= []
list_l= []
for f in file_list:
     if 'Data_' in f or 'Import' in f or 'Data.' in f:
        df = pl.read_excel(f,  read_csv_options={"infer_schema_length": None})
        list_f.append(f.split("\\",-1)[1])
        list_l.append(df.height)

df_to_import = pd.DataFrame(zip(list_f, list_l), columns=('file', 'length'))
exclude_list = ['Data_20220812_20220821.xlsx']
df_to_import = df_to_import[~df_to_import['file'].isin(exclude_list)]

# File Data_20220812_20220821 Đã nằm trong Data nên không cần import nó 
# Mà chỉ cần import file Data
# File cuối cùng còn thiếu là missingDoc

In [ ]:
df_to_import[11:]

## Function

In [ ]:
def run_import_akc_v2(_file_to_import):

    import polars as pl

    # source_ = 'Data_20220822_20220828'
    # source_ = 'Data_20230109_20230115'
    source_ = _file_to_import
    def proc_etl_pl(source_):
        df_pl = pl.read_excel(r"C:/Users/Admin/OneDrive/FRT/6. Stored Data/AkachainMember/Data/" + source_ , read_csv_options={"infer_schema_length": 0})
        df_pl = df_pl.with_columns(pl.col('Ngày tạo').str.strptime(pl.Date, fmt = '%Y-%m-%d').cast(pl.Date))
        df_pl = df_pl.with_columns(pl.col('Ngày giao dịch').str.strptime(pl.Date, fmt = '%Y-%m-%d').cast(pl.Date))
        df_pl = df_pl.with_columns(pl.col('Điểm phát sinh').cast(pl.Int64))
        df_pl = df_pl.with_columns(pl.col('Điểm xếp hạng quy đổi').cast(pl.Int64))
        df_pl = df_pl.with_columns(pl.col('Số tiền mua').cast(pl.Float64)).with_columns(pl.col('Số tiền mua').cast(pl.Int64))

        df_pl = df_pl.rename({
                    'Mã thành viên'             : 'customer_code_akc',
                    'Ngày tạo'                  : 'created_date',
                    'Ngày giao dịch'            : 'transaction_date',
                    'Lý do'                     : 'transaction_type_name',
                    'Mã giao dịch'              : 'transaction_code',
                    'Số tiền mua'               : 'amount', 
                    'Điểm xếp hạng quy đổi'     : 'point_ranking',  
                    'Điểm phát sinh'            : 'point',
                    })
        df_pl = df_pl.drop('Điểm đổi quà đã sử dụng')
        df_pl = df_pl.with_columns(pl.lit(datetime.now()).alias('time_insert'))
        df_pl = df_pl.with_columns(pl.lit(source_).alias('source'))
        return df_pl

    df_akc = proc_etl_pl(source_)

    length_df_begin = df_akc.height
    # Step 1: ETL from source
    min_date_filter = df_akc['created_date'].min()
    print(f'Min date filter is {min_date_filter}')

    # Information of  server
    USER_NAME   = 'ecom_user'
    PW          = 'Ec0m@12345'
    SERVER      = '118.69.201.34'
    DATABASE    = 'FLC_SHOP_SALES_DATAWAREHOUSE'
    conn_dw     = f"mssql://{USER_NAME}:{PW}@{SERVER}/{DATABASE}?driver=SQL+Server"


    display(df_akc.head(1))


    print('|| Step 2:')
    # Step 2: 
    # Những hiểu biết trước thì 1 mã AKC chỉ có 1 Mã Customer Key nên không cần check lại
    # Nhưng thực tế bảng Loyalty của AKC thì có trường hợp 1 akc_cus_code = 2 customer_key

    # Lấy ra danh sách Mã thành viên AKC chưa có trong DIM
    query_dim_cus = """SELECT DISTINCT key_id,customer_code_akc, completed_flg FROM D_ICT_AKACHAIN_LOYALTY_CUS_MAPPING_v2"""
    dim_cus = pl.read_sql(query_dim_cus, conn_dw)

    # Danh sách mã thành viên chưa có DIM Cus
    df_get_cus_info = df_akc.join(dim_cus, on = 'customer_code_akc', how='left', )

    # Print stats:
    display(df_get_cus_info\
        .groupby('completed_flg')\
        .agg(pl.n_unique('customer_code_akc').alias('count_code')))

    # Filter chỉ lấy những customer nào chưa completed
    list_inv = df_get_cus_info\
                    .filter( (pl.col('transaction_type_name')=='Invoice') & (pl.col('completed_flg')!=1) )\
                    .select(pl.col('transaction_code')).unique() # Danh sách inv sẽ search
    list_inv = list_inv['transaction_code'].to_list()
    print(f"Total List to get: {len(list_inv):,.0f}")
    list_cus_inv_out = pd.DataFrame()
    num_part = math.ceil(len(list_inv) /15_000) # Chia tối đa read 15,000 records/lần

    print("Get list Customer with Invoice")
    for i in range(1,num_part+1):
        list_ = list_inv[(i-1)*15_000 : min(i * 15_000, len(list_inv))]
        num_record = min(i * 15_000, len(list_inv))
        print(f"|_{i} | {num_record:,.0f} | {num_record / len(list_inv):,.1%}")
        list_ = ','.join(list_)

        query_read = f"""SELECT DISTINCT a.CUSTOMER_KEY, a.INVOICE_HEADER 
                FROM VF_CID_SALES_TRANSACTION_ICT a
                LEFT JOIN (SELECT value as 'INVOICE_HEADER' FROM STRING_SPLIT( '{list_}' , ',')) b
                    ON a.INVOICE_HEADER = b.INVOICE_HEADER
                WHERE b.INVOICE_HEADER IS NOT NULL"""

        # Tạm thời read bẳng pandas vì connectorx chưa tìm ra cách connect tới azure sql server
        list_cus_inv = pd.read_sql(query_read, azr_engine)
        list_cus_inv_out = pd.concat([list_cus_inv_out, list_cus_inv])

    # Do test trên SQL thì nếu left join luôn DIM customer thì sẽ chậm hơn việc SELECT IN list customer_key
    # Nên chia làm 2 step : 1 là lấy ra danh sách cus, 2 là lấy ra thông tin cus

    print('|| Step 3: Get customer information')

    # Step 3: Get customer Information 
    n_cus_scan = list_cus_inv_out.CUSTOMER_KEY.nunique()

    list_scan_cus = list_cus_inv_out.CUSTOMER_KEY.unique().tolist()
    list_cus_info_out = pd.DataFrame()
    num_part_cus = math.ceil(len(list_scan_cus) /15_000) # Chia tối đa read 15,000 records/lần

    print("Step 3: Get list Customer information with CUSTOMER_KEY")
    print("Total customer to scan: ", f"{(n_cus_scan):,.0f}")

    for i in range(1, num_part_cus+1):
        list_ = list_scan_cus[(i-1)*15_000 : min(i * 15_000, len(list_scan_cus))]
        num_record = min(i * 15_000, len(list_scan_cus))
        print(f"|_{i} | {num_record:,.0f} | {num_record / len(list_scan_cus):,.1%}")
        list_ = ','.join(str(v) for v in list_)
        query_read = f"""SELECT CUSTOMER_KEY, CUSTOMER_NAME, CUSTOMER_ADDRESS, CUSTOMER_PHONE
                        FROM VD_CID_CUSTOMER_ICT
                        WHERE CUSTOMER_KEY IN 
                            (SELECT value as 'CUSTOMER_KEY' FROM STRING_SPLIT( '{list_}' , ',')) 
                    """   
        # Tạm thời read bẳng pandas vì connectorx chưa tìm ra cách connect tới azure sql server
        list_cus_info = pd.read_sql(query_read, azr_engine)
        list_cus_info_out = pd.concat([list_cus_info_out, list_cus_info])

    # Check diff
    print(f"Total cus scan: {list_cus_inv_out.CUSTOMER_KEY.nunique():,.0f}")
    print(f"Total cus result: {list_cus_info_out.CUSTOMER_KEY.nunique():,.0f}")
    print(f"Diff : {list_cus_info_out.CUSTOMER_KEY.nunique() - list_cus_inv_out.CUSTOMER_KEY.nunique() :,.0f}")



    print('|| Step 4:')

    # Step 4: Clean lại file và import to Dim Customer
    df_import_cus = list_cus_inv_out.merge(list_cus_info_out, on ='CUSTOMER_KEY', how = 'left', indicator=True)
    df_import_cus = pl.from_pandas(df_import_cus)
    df_import_cus = df_import_cus.with_columns(pl.when(pl.col('_merge') == 'both').then(1).otherwise(0).alias('completed_flg'))
    df_import_cus = df_import_cus.select(
                                pl.col('CUSTOMER_KEY').alias('customer_key'), 
                                pl.col('INVOICE_HEADER').cast(pl.Utf8).alias('transaction_code'), 
                                pl.col('CUSTOMER_NAME').alias('customer_name'), 
                                pl.col('CUSTOMER_ADDRESS').alias('customer_address'),  
                                pl.col('CUSTOMER_PHONE').alias('customer_phone'), 
                                pl.col('completed_flg'), )

    df_import_cus = df_import_cus.join(df_get_cus_info\
        .filter((pl.col('transaction_type_name')=='Invoice') & (pl.col('completed_flg')!=1))\
        .select(pl.col('customer_code_akc'), pl.col('transaction_code'))
        , how='outer'
        , on='transaction_code'
    )

    # Có những document không tìm được trong db prod nên loại bỏ
    # display(df_import_cus.filter(pl.col('customer_code_akc') == '04c21fe9-e07c-4730-9d92-38c6621b067d'))
    # display(df_import_cus.filter(pl.col('customer_key').is_null()))
    df_import_cus = df_import_cus.filter(~pl.col('customer_key').is_null())


    df_import_cus = df_import_cus.drop('transaction_code').unique()
    df_import_cus = df_import_cus.join(
                    df_get_cus_info.select('customer_code_akc').unique()
                    , how='outer'
                    , on='customer_code_akc'
    )
    df_import_cus = df_import_cus.with_columns(pl.col('completed_flg').fill_null(0))
    stats_null = df_import_cus.groupby('completed_flg').agg(pl.n_unique('customer_code_akc').alias('total_cus'))
    display(stats_null)
    print(
    f"""Total cus import: {df_import_cus.select('customer_code_akc').n_unique():,.0f}
    Total cus Scan: {stats_null['total_cus'].sum():,.0f}
    ## Diff: {df_import_cus.select('customer_code_akc').n_unique() - stats_null['total_cus'].sum():,.0f}
    """)
    # Reorder columns
    df_import_cus = df_import_cus[['customer_key', 'customer_code_akc', 'customer_name', 'customer_address', 'customer_phone', 'completed_flg']]
    df_import_cus = df_import_cus.with_columns(pl.lit(datetime.now()).alias('time_insert'))
    df_import_cus = df_import_cus.with_columns(pl.lit(source_).alias('source'))

    # Trước khi import thì loại bỏ những customer đã có status = 1 trong db
    len_before = len(df_import_cus)
    print(f"Before remove completed: {len_before:,.0f}")
    df_import_cus = df_import_cus.filter(
                        ~pl.col('customer_code_akc').is_in(dim_cus.filter(pl.col('completed_flg') == True)['customer_code_akc']
                        ))

    dim_cus.filter(pl.col('completed_flg') == True).select('customer_code_akc').unique()
    print(f"After remove completed: {len(df_import_cus):,.0f} ({len(df_import_cus)-len_before:,.0f})")

    # Phải remove thêm 2 lần nữa

    # Remove lần 1
    # Nếu before là 0 , after là 0 thì không cần làm gì

    df_ba = df_import_cus.select('customer_code_akc', 'completed_flg').join(dim_cus, on = 'customer_code_akc', how='left')

    df_import_cus = df_import_cus.filter(
                        ~pl.col('customer_code_akc').is_in(
                            df_ba.filter( 
                                        (pl.col('completed_flg') == 0) & 
                                        (pl.col('completed_flg_right') ==  0)
                                        )['customer_code_akc']
                        ))

    # Remove lần 2
    # Nếu before là 0 , after là 1 thì update,
    df_update = df_import_cus.join(dim_cus, on = 'customer_code_akc', how='left')
    df_update = df_update.filter((pl.col('completed_flg') == 1) & (pl.col('completed_flg_right') == 0)).to_pandas()
    if len(df_update) > 0:
        values_list = []
        for index, row in df_update.iterrows():
            values_list.append(
                (row['customer_key']
                , row['customer_code_akc']
                , row['customer_name']
                , row['customer_address']
                , row['customer_phone']
                , row['completed_flg']
                , row['time_insert']
                , row['source']
                , row['key_id'])
                )
        import pyodbc

        # connect to SQL Server database
        cnxn = pyodbc.connect(SaleMart)
        cursor = cnxn.cursor()

        # define update query
        update_query = """
                    UPDATE D_ICT_AKACHAIN_LOYALTY_CUS_MAPPING_v2
                    SET 
                        customer_key = ?, 
                        customer_code_akc = ?,
                        customer_name = ?,
                        customer_address = ?,
                        customer_phone = ?,
                        completed_flg = ?,
                        time_insert = ?,
                        source = ?
                    WHERE key_id = ?
                    """

        # execute update query for each set of values
        for values in values_list:
            cursor.execute(update_query, values)

        # commit changes to database
        cnxn.commit()

        # close cursor and connection
        cursor.close()
        cnxn.close()

        df_import_cus = df_import_cus.filter(
                            ~pl.col('customer_code_akc').is_in(
                                df_update['customer_code_akc'].to_list()
                            ))

    df_import_cus.to_pandas().to_sql("D_ICT_AKACHAIN_LOYALTY_CUS_MAPPING_v2", getEngine(SaleMart), if_exists='append', index=False)

    # Viết lại tiếp 1 step nữa là delete những records nào mà complete thay đổi khi chạy lại


    print('|| Step 5:')
    # Step 5: Import df vào bảng, tuy nhiên cần check xem có trùng ko
    # Read data after datefilter
    min_date_filter_ = df_akc['created_date'].min() - timedelta(days=5)
    print(f"Check data date after: {min_date_filter}")

    query_r_history = f"""SELECT 
                    DISTINCT created_date, transaction_date, transaction_code
                    FROM F_ICT_AKACHAIN_LOYALTY_v2 
                    WHERE created_date >=  '{str(min_date_filter_)}'
                    """
    history = pl.read_sql(query_r_history, conn_dw)
    history = history.with_columns(pl.lit(1).alias('flg'))
    if len(history)>0:
        display(history.sample(5))
    else:
        print("Df history is null")

    output = df_akc.join(history, on=['created_date', 'transaction_date', 'transaction_code'], how='left')

    print(f"Number of duplicate: {len(output.filter(pl.col('flg')==1)):,.0f}")
    print(f"Number of not duplicate: {len(output.filter(pl.col('flg')!=1)):,.0f}")

    # Filter final
    output = output.filter(pl.col('flg')!=1).drop('flg')
    output.to_pandas().to_sql("F_ICT_AKACHAIN_LOYALTY_v2", getEngine(SaleMart), if_exists='append', index=False)

    print(f"Done import ")
    print(f"Length df begin: {length_df_begin:,.0f}")
    print(f"Length df final: {output.height:,.0f}")
    print(f"Diff length df {length_df_begin - output.height:,.0f}")

In [ ]:
for f in df_to_import['file'][21:47]:
    print('===========================================')
    print(f'Run file is {f}')
    run_import_akc_v2(f)

In [ ]:
df_to_import

In [ ]:
# Câu để check 2 table có diff không

"""
  SELECT DISTINCT a.customer_code_akc , b.customer_code_akc
  FROM F_ICT_AKACHAIN_LOYALTY_v2 a
  LEFT join D_ICT_AKACHAIN_LOYALTY_CUS_MAPPING_v2  b
  ON a.customer_code_akc = b.customer_code_akc
  WHERE b.customer_code_akc is null

"""


In [ ]:
df_to_import[1:5]

In [ ]:
df

In [ ]:
# Check lại trước khi chạy full log
import os, glob
import polars as pl
import pandas as pd
dir = r"C:/Users/Admin/OneDrive/FRT/6. Stored Data/AkachainMember/Data/"
file_list = glob.glob(os.path.join(dir, "*.xlsx"))
# file_list = [i for i in file_list if 'Data_' in i or 'Import' in i]
list_f= []
list_l= []
for f in file_list:
     if 'Data_' in f or 'Import' in f or 'Data.' in f:
        df = pl.read_excel(f,  read_csv_options={"infer_schema_length": None})
        list_f.append(f.split("\\",-1)[1])
        list_l.append(df.height)

df_to_import = pd.DataFrame(zip(list_f, list_l), columns=('file', 'length'))
exclude_list = ['Data_20220812_20220821.xlsx']
df_to_import = df_to_import[~df_to_import['file'].isin(exclude_list)]

# File Data_20220812_20220821 Đã nằm trong Data nên không cần import nó 
# Mà chỉ cần import file Data
# File cuối cùng còn thiếu là missingDoc

# DONE


In [ ]:
## RECHECK DATA

source_ = 'AKC_Revenue_Detailed_30.08-13.09_ICT'
df = pd.read_csv(r"C:/Users/Admin/OneDrive/FRT/6. Stored Data/AkachainMember/Weeklycheck/" + source_ + ".csv")


In [ ]:
df

In [ ]:
list_inv = df['OriginalTransCodeOfBu'].drop_duplicates()
num_part = math.ceil(len(list_inv) /15000)

df_out = pd.DataFrame()
print(f"Number of records = {len(list_inv)}")
for i in range(1,num_part+1):
    list_ = list_inv[(i-1)*15000 : min(i * 15000, len(list_inv))]
    print(i, min(i * 15000, len(list_inv)), len(list_))
    list_ = ','.join(list_.astype(str))
    query_read = f"""SELECT *
                    FROM [dbo].[VF_CID_SALES_TRANSACTION_ICT]
                    WHERE 1=1
                        AND [W_DELETE_FLG] = 'N'
                        AND IS_PROJECT = 'N'
                        AND ORDER_TYPE NOT IN (N'Xuất trả NCC', N'Bán nội bộ', 'SO Service')
                        AND CUSTOMER_KEY NOT IN (1480086, 24235209, 5327471, 16957226) -- Long Châu, FPT
                        AND SHOP_KEY NOT IN (1667,648,10720,903,1224,1466,1552)
                        AND INVOICE_HEADER IN ( {list_} )"""
    df_read = pd.read_sql(query_read, azr_engine)
    df_out = pd.concat([df_out, df_read])


In [ ]:
df_out[df_out['INVOICE_HEADER'] == ]

In [ ]:
df_out.to_csv(r"C:/Users/Admin/OneDrive/FRT/6. Stored Data/AkachainMember/Weeklycheck/OutCheck.csv")

In [ ]:
## Import Danh sách khách hàng Inactive

source_ = 'List_Inactive_Member_221004'
df_inactive = pd.read_excel(r"C:/Users/Admin/OneDrive/FRT/6. Stored Data/AkachainMember/Data/" + source_ + ".xlsx")
df_inactive.to_sql("D_ICT_AKACHAIN_LOYALTY_CUS_INACTIVE", getEngine(SaleMart), if_exists='append', index=False)


In [ ]:
## Check lại data 34 và Azure
f_date = '2022-10-01'
t_date = '2022-10-10'



In [ ]:
# Danh sách inactive
query_inactive = f"""SELECT   [Mã thành viên], CUSTOMER_KEY
  FROM D_ICT_AKACHAIN_LOYALTY_CUS_MAPPING
  WHERE [Mã thành viên] IN (SELECT [Mã thành viên] FROM D_ICT_AKACHAIN_LOYALTY_CUS_INACTIVE)
                """
df_inactive = pd.read_sql(query_inactive, getEngine(SaleMart))


In [ ]:
query_read_34 = f"""SELECT  a.[Mã thành viên]
                    , b.CUSTOMER_KEY
                    ,[Ngày tạo]
                    ,[Ngày giao dịch]
                    ,[Lý do]
                    ,[Mã giao dịch]
                    ,[Số tiền mua]
                    ,[Điểm xếp hạng quy đổi]
                    ,[Điểm phát sinh]
                FROM [FLC_SHOP_SALES_DATAWAREHOUSE].[dbo].[F_ICT_AKACHAIN_LOYALTY] a
                LEFT JOIN D_ICT_AKACHAIN_LOYALTY_CUS_MAPPING b
                    ON a.[Mã thành viên] = b.[Mã thành viên]
                WHERE [Lý do] = 'Invoice'
                AND [Ngày giao dịch] BETWEEN '{f_date}' AND '{t_date}'
                """
df_aka = pd.read_sql(query_read_34, getEngine(SaleMart))


In [ ]:
# Đọc data từ Azure
query_read_az = f"""SELECT INVOICE_HEADER, CUSTOMER_KEY, SUM(LIST_AMT_AF_VAT) AS AMOUNT
    FROM VF_CID_SALES_TRANSACTION_ICT
    WHERE TRANSACTION_TYPE = 'Sales Invoice'
        AND  [W_DELETE_FLG] = 'N'
        AND CUSTOMER_KEY NOT IN (1480086, 24235209, 5327471, 16957226) -- Long Châu, FPT

        AND CUSTOMER_KEY NOT IN (15993818) -- Đơn hàng truy thu
--        AND PRODUCT_KEY NOT IN (SELECT PRODUCT_KEY      FROM VD_CID_PRODUCT WHERE W_DATASOURCE_ID = 'ICT' AND [PRODUCT_CATEGORY_NAME]   IN  ('DV', N'SUBSIDY')) -- Thẻ cào
        AND  CONVERT(DATE, CONVERT(VARCHAR(10), [DATE_KEY])) BETWEEN '{f_date}' AND '{t_date}' 
        GROUP BY INVOICE_HEADER, CUSTOMER_KEY  """
df_az = pd.read_sql(query_read_az, azr_engine)
df_az['AMOUNT'] = df_az['AMOUNT'].astype(int)

# Loại bỏ những khách inactive
df_az = df_az[~df_az['CUSTOMER_KEY'].isin(df_inactive['CUSTOMER_KEY'].values)]

In [ ]:
# Check duplicate giao dịch
c_dup = df_aka.groupby('Mã giao dịch').agg(countL =('Mã giao dịch','count'))
c_dup = c_dup[c_dup['countL']>1] 
n_dup = len(c_dup)
print(f"Số lượng line dup = {n_dup}")
display(c_dup)

In [ ]:
display(df_aka.sample(3), df_az.sample(3))

In [ ]:
df_aka_agg = df_aka.groupby(['Mã giao dịch', 'CUSTOMER_KEY'], as_index=False).agg(amt_aka = ('Số tiền mua', 'sum'))
df_aka_agg['Mã giao dịch'] = df_aka_agg['Mã giao dịch'].astype(int)
df_aka_agg

In [ ]:
df_merge = df_az.merge(df_aka_agg, how='outer', left_on=['INVOICE_HEADER'], right_on=['Mã giao dịch'], suffixes=['_Az', '_AKA'])
df_merge['AMOUNT'] = df_merge['AMOUNT'].fillna(0)
df_merge['amt_aka'] = df_merge['amt_aka'].fillna(0)

df_merge['DiffAmt'] = df_merge['AMOUNT'] - df_merge['amt_aka']
df_diff = df_merge[(df_merge['CUSTOMER_KEY_Az']!=df_merge['CUSTOMER_KEY_AKA']) | (df_merge['DiffAmt']>1000)| (df_merge['DiffAmt']<-1000)]

_so_line_diff =len(df_diff)
display(len(df_diff))

In [ ]:
# Invoice_Header is na
inv_na = df_diff[(df_diff['INVOICE_HEADER'].isna())]
print(f"Số INVOICE_HEADER is NA = {len(inv_na)} | AMT = {_convert_decimal(df_diff['amt_aka'].sum(),0)}")
display(inv_na.sample(3))
gd_na = df_diff[(df_diff['Mã giao dịch'].isna())]
print(f"Số Mã gd is NA = {len(gd_na)} | AMT = {_convert_decimal(df_diff['AMOUNT'].sum(),0)}")
display(gd_na.sample(3))


In [ ]:
diff_amt = df_diff[~((df_diff['INVOICE_HEADER'].isna()) | (df_diff['Mã giao dịch'].isna()))]
print(f"Số Mã gd is NA = {len(diff_amt)} | AMT = {_convert_decimal(diff_amt['DiffAmt'].sum(),0)}")
display(diff_amt.sample(3))


In [ ]:
# Xem AMT = 0 thi thuoc cus nao
df_diff[(df_diff['amt_aka'].isna()) | (df_diff['amt_aka']==0)].groupby('CUSTOMER_KEY_Az').agg(countInv =('INVOICE_HEADER','count')).sort_values(by='countInv', ascending=False)

In [ ]:
df_diff.to_csv(r"C:/Users/Admin/OneDrive/FRT/6. Stored Data/AkachainMember/Weeklycheck/OutCheck.csv")

In [ ]:
## Summary Cho a Chương
import pandas as pd
import numpy as np
import glob, os 
import pandas_profiling as pp
from HelperMaster import draw_hist
dir = r"C:\Users\Admin\OneDrive\FRT\6. Stored Data\AkachainMember\Summary\Data"

In [ ]:
list_file_tieu = []
for file in os.listdir(dir):
    if file.endswith(".csv"):
        if 'tiêu' in file and 'ngày' in file:
            file_dir = os.path.join(dir, file)
            print(file)
            list_file_tieu.append(file_dir)
tieudiem = pd.concat([pd.read_csv(i) for i in list_file_tieu])

list_file_tich = []
for file in os.listdir(dir):
    if file.endswith(".csv"):
        if 'tích' in file and 'ngày' in file:
            print(file)
            file_dir = os.path.join(dir, file)
            list_file_tich.append(file_dir)
            # display(pd.read_csv(file_dir))
tichdiem = pd.concat([pd.read_csv(i) for i in list_file_tich])

tieudiem['Month'] = pd.to_datetime(tieudiem['DATE']).dt.month

In [ ]:
len(tieudiem), len(tichdiem)

In [ ]:
pp.ProfileReport(tieudiem)

In [ ]:
tieudiem.groupby(['Month','GRANTED_MERCHANT_NAME', 'USED_MERCHANT_NAME']).agg(
    countMemberID = ('MEMBER_ID', 'nunique'),
    countMemberCode = ('MEMBER_CODE', 'nunique'),
    totalPoint = ('TOTAL_FGOLD_AMOUNT_IN_DAY', 'sum'),
)


In [ ]:
tieudiem

In [ ]:
# Def check
dir = r"C:\Users\Admin\OneDrive\FRT\6. Stored Data\AkachainMember\Summary\Data"
list_M = ['9', '10']

df_tieu_details = pd.concat([pd.read_csv(dir + f"/T{i} - Báo cáo tiêu điểm chi tiết trên từng giao dịch.csv") for i in list_M])
df_tieu_summary = pd.concat([pd.read_csv(dir + f"/T{i} - Báo cáo tiêu điểm trên từng thành viên theo ngày.csv") for i in list_M])
df_tieu_details['DATE'] = pd.to_datetime(df_tieu_details['USED_TIME_GMT'], format="%Y-%m-%d").dt.floor('d')
df_tieu_summary['DATE'] = pd.to_datetime(df_tieu_summary['DATE'])

df_tich_details = pd.concat([pd.read_csv(dir + f"/T{i} - Báo cáo tích điểm chi tiết trên từng giao dịch.csv") for i in list_M])
df_tich_summary = pd.concat([pd.read_csv(dir + f"/T{i} - Báo cáo tích điểm trên từng thành viên theo ngày.csv") for i in list_M])
df_tich_details = df_tich_details.drop(columns=['GRANT_TYPE', 'REASON', 'USAGE_PRIORITY','PARTNER_POINT_AMOUNT','POINT_EXCHANGE_RATE','CURRENCY_EXCHANGE_RATE',])
df_tich_summary.drop(columns=['NAME', 'PHONE'],inplace=True)
df_tich_summary.DATE = pd.to_datetime(df_tich_summary.DATE)

df_tich_details[['VALID_DATE_GMT', 'EXPIRY_DATE_GMT', 'BUSINESS_TIME_GMT', 'CREATED_TIME_GMT']] = df_tich_details[['VALID_DATE_GMT', 'EXPIRY_DATE_GMT', 'BUSINESS_TIME_GMT', 'CREATED_TIME_GMT']].apply(pd.to_datetime)
 


In [ ]:
## Valid_date và business date bằng nhau
df_tich_details['Diff_time_create_business'] =  np.floor((df_tich_details.CREATED_TIME_GMT - df_tich_details.BUSINESS_TIME_GMT).dt.total_seconds()/3600)
df_tich_details['Diff_time_valid_business'] =  np.floor((df_tich_details.VALID_DATE_GMT - df_tich_details.BUSINESS_TIME_GMT).dt.total_seconds())
df_tich_details.sort_values(by='Diff_time_create_business', ascending=False)[:10]
# df_tich_details['Diff_time'].hist()

In [ ]:
df_tich_summary[df_tich_summary.USER_ADDRESS== 'f1306a1fc48778c4f8046eaf810bed8fd8e8f3']

In [ ]:
# pp.ProfileReport(df_tich_details)
# df_tich_summary
df_tich_details

In [ ]:
df_dt_group = df_tieu_details.groupby(
    ['MEMBER_CODE', 'USED_MERCHANT_NAME', 'GRANTED_MERCHANT_NAME', 'DATE'], as_index=False)\
    .agg(sumToken = ('TOKEN_AMOUNT', 'sum'), frequency = ('ORDER_CODE', 'nunique'))
df_dt_group.head(3)

In [ ]:
df_tieu_summary.head(3)

In [ ]:
df_tieu_out = df_tieu_summary[['DATE', 'MEMBER_CODE','USED_MERCHANT_NAME','GRANTED_MERCHANT_NAME', 'TOTAL_TRANSACTION_IN_DAY', 'TOTAL_FGOLD_AMOUNT_IN_DAY']].merge(
    df_dt_group,how='outer',
    left_on=['DATE','MEMBER_CODE','USED_MERCHANT_NAME', 'GRANTED_MERCHANT_NAME'],
    right_on=['DATE','MEMBER_CODE','USED_MERCHANT_NAME', 'GRANTED_MERCHANT_NAME'])
df_tieu_out['Diff_Token'] = df_tieu_out.TOTAL_FGOLD_AMOUNT_IN_DAY.fillna(0) - df_tieu_out.sumToken.fillna(0)
df_tieu_out['Diff_Trans'] = df_tieu_out.TOTAL_TRANSACTION_IN_DAY.fillna(0) - df_tieu_out.frequency.fillna(0)

df_tieu_out.to_excel(dir + "/Output.xlsx", index=False)
df_tieu_out.head(2)

In [ ]:
# df_summary.head(3)c
# df_dt_group.head(3)8
df_out.head(3)

In [ ]:
df_tieu_summary[df_tieu_summary.MEMBER_CODE=='db88b1ac-8a7f-4996-b525-5528afb71c6b']

In [ ]:
df_tieu_summary.groupby('GRANTED_MERCHANT_NAME')['TOTAL_FGOLD_AMOUNT_IN_DAY'].sum()

In [ ]:
# Check điểm ngày 16/1 export
import glob
import pandas as pd 
dir = r"C:\Users\Admin\OneDrive\FRT\6. Stored Data\AkachainMember\Check_Maphieu"
file_list = glob.glob(dir + "/ICT_transaction*.xlsx")

excl_list = []

for file in file_list:
    excl_list.append(pd.read_excel(file))
# create a new dataframe to store the
# merged excel file.
excl_merged = pd.DataFrame()
 
for excl_file in excl_list:
    # appends the data into the excl_merged
    # dataframe.
    excl_merged = excl_merged.append(
      excl_file, ignore_index=True)

In [ ]:
# Do các giao dịch Rollback ghi nhận điểm sai nên cần itn1h lại
excl_merged['Point_cal'] = np.where(excl_merged['Mã giao dịch'].str.startswith('Rollback',na=False), excl_merged['Điểm xếp hạng'], excl_merged['Điểm đổi quà'])
excl_merged[excl_merged['Mã giao dịch'].str.startswith('Rollback',na=False)]

In [ ]:
x = excl_merged.groupby('Mã thành viên', as_index=False).agg(totalP = ('Point_cal','sum'))



In [ ]:
x.totalP.sum()

In [ ]:
excl_merged

In [ ]:
x[x['Mã thành viên'] == '01378e1d-e3d0-499b-9492-ed93198516fd']

In [ ]:
excl_merged[excl_merged['Mã thành viên']=='01378e1d-e3d0-499b-9492-ed93198516fd'].sort_values(by = 'Ngày giao dịch', ascending=False)

In [ ]:
list_roll_cus = excl_merged[excl_merged['Mã giao dịch'].str.startswith('Rollback',na=False)]['Mã thành viên'].values

In [ ]:
## Export balance cho anh Chương
data_ex = excl_merged[['Mã thành viên', 'Mã giao dịch','Mã giao dịch gốc', 'Điểm xếp hạng', 'Điểm đổi quà', 'Tổng số tiền tính toán', 'Ngày tạo']]
data_ex['Ngày tạo'] = data_ex['Ngày tạo'].dt.date
data_ex['Count_ex'] = 1
data_ex['Mã giao dịch'] = data_ex['Mã giao dịch'].astype(str)
data_ex = data_ex[data_ex['Ngày tạo'] < dt.date(2023,1,1)]
data_ex['Point_cal'] = np.where(data_ex['Mã giao dịch'].str.startswith('Rollback',na=False), data_ex['Điểm xếp hạng'], data_ex['Điểm đổi quà'])

data_mir = pd.read_sql("""SELECT [Mã thành viên], [Lý do], [Mã giao dịch], [Số tiền mua], [Điểm phát sinh], [Ngày tạo] FROM F_ICT_AKACHAIN_LOYALTY WHERE [Ngày tạo] < '2023-01-01' AND [Lý do] = 'Migration' """, getEngine(SaleMart))
data_mir = data_mir[data_mir['Điểm phát sinh']!=0]
data_mir = data_mir.groupby('Mã thành viên', as_index=False).agg(totalMir = ('Điểm phát sinh', 'sum'))


In [ ]:
x = data_ex.groupby('Mã thành viên', as_index=False).agg(totalPoint = ('Point_cal','sum'))
x = x.merge(data_mir[['Mã thành viên','totalMir']], on='Mã thành viên', how='outer')
x.fillna(0, inplace=True)
x['FinalPoint'] = x['totalPoint'] + x['totalMir']
# x.to_excel(dir + "/Check.xlsx")

In [ ]:
x

In [ ]:
x.FinalPoint.sum()

In [ ]:
x[x.FinalPoint != 0 ].to_excel(dir + "/Exp.230210.xlsx")

In [ ]:
x[x['Mã thành viên'] == '01378e1d-e3d0-499b-9492-ed93198516fd']

In [ ]:
excl_merged[excl_merged['Mã thành viên']=='4d6fa515-02c5-4c95-96b3-e1af92f900fb']

In [ ]:
## Export lại Data
dir  = r"C:\Users\Admin\OneDrive\FRT\6. Stored Data\AkachainMember\Export Balance 221231"

file_list = glob.glob(dir + "/22_*.xlsx")

excl_list = []

for file in file_list:
    excl_list.append(pd.read_excel(file))
# create a new dataframe to store the
# merged excel file.
excl_merged = pd.DataFrame()
 
for excl_file in excl_list:
    # appends the data into the excl_merged
    # dataframe.
    excl_merged = excl_merged.append(
      excl_file, ignore_index=True)
    


In [ ]:
excl_merged.to_csv(dir + "/OutputMerge_2022.csv")

In [ ]:
df_merge = pd.read_csv(dir + "/OutputMerge_2022.csv")

In [ ]:
_query =  """  SELECT [Ngày tạo], [Lý do], [Mã giao dịch], [Điểm phát sinh]
  FROM [FLC_SHOP_SALES_DATAWAREHOUSE].[dbo].[F_ICT_AKACHAIN_LOYALTY]
  WHERE [Ngày tạo] < '2023-01-01' """
df_34 = pd.read_sql(_query, getEngine(SaleMart))

### Ngày 20/03/2023: Danh  request cách đối chiếu giữa 34 và az

In [ ]:
f_date = '2023-02-01'
t_date = '2023-02-28'
import polars as pl 


df_34 = pd.read_sql(f"""SELECT 
                        [Mã thành viên] AS 'MEMBER_CODE_AKC',
                        [Ngày tạo] AS 'CREATED_DATE',
                        [Ngày giao dịch] AS 'TRANSACTION_DATE',
                        [Lý do] AS 'POINT_INFORMATION',
                        [Mã giao dịch] AS 'DOCENTRY',
                        [Số tiền mua] AS 'AMOUNT',
                        [Điểm xếp hạng quy đổi] AS 'POINT_RANKING',
                        [Điểm phát sinh] AS 'POINT'
                        FROM F_ICT_AKACHAIN_LOYALTY 
                        WHERE [Ngày giao dịch] BETWEEN '{f_date}' AND '{t_date}' """,  getEngine(SaleMart))
df_34= df_34.rename(columns=str.lower)
df_34.head(5)

In [ ]:
df_az = pd.read_sql(f"""SELECT  *
FROM [dbo].[VF_CID_AKC_TRANSACTION_LOYALTY_ICT]
WHERE CAST(CREATED_DATE AS DATE)  BETWEEN '{f_date}' AND '{t_date}' """,  azr_engine)
df_az= df_az.rename(columns=str.lower)

df_az.head(5)

In [ ]:
# Diff Inv
inv_34  = df_34[df_34['point_information'] == 'Invoice']
inv_34 = inv_34.groupby(['member_code_akc', 'docentry']).agg(
    point_34 = ('point','sum'),
    amt_34 = ('amount', 'sum')
)
inv_az  = df_az[df_az['point_information'] == 'Invoice']
inv_az = inv_az.rename(columns={'invoice_header':'docentry'}).groupby(['member_code_akc', 'docentry']).agg(
    point_az = ('invoice_point','sum'),
    amt_az = ('list_amount_af_vat', 'sum')
)
display(inv_34.head(3))
display(inv_az.head(3))
print(inv_az.count(), inv_az.count())


In [ ]:
merge_inv = inv_34.merge(inv_az, how='outer', left_index=True, right_index=True)
# merge_inv.fillna(0, inplace=True)
merge_inv['diff_point'] = merge_inv.point_34.fillna(0) - merge_inv.point_az.fillna(0)
merge_inv['diff_amt'] = merge_inv.amt_34.fillna(0) - merge_inv.amt_az.fillna(0)
merge_inv['diff_point_flg'] = np.where(merge_inv.point_az != merge_inv.point_34 , True,False)
merge_inv['diff_amt_flg'] = np.where(merge_inv.amt_34 !=  merge_inv.amt_az , True,False)

print(merge_inv[['diff_point_flg', 'diff_amt_flg']].sum())
print(merge_inv.diff_point.abs().sum() )
print(merge_inv.diff_amt.abs().sum() )


In [ ]:
merge_inv[merge_inv.diff_point_flg==True]

In [ ]:
merge_inv[(merge_inv.diff_amt_flg==True) & (merge_inv.diff_point_flg==False)].sort_values('diff_amt',ascending=False)

In [ ]:
# 23/03/2023: Export chạy thập phân vị cho a chương số điểm lũy kế từng khách
from HelperMaster import cal_percentile
df = pd.read_csv(r'C:\Users\Admin\OneDrive\FRT\6. Stored Data\AkachainMember\230323_export.csv')
df.drop(columns='IsGrandTotalRowTotal', inplace=True)
df = df[~df['Mã thành viên'].isna()]
df = df[df['Số_điểm_Lũy_kế']!=0]

df

In [ ]:
df[>7890.0.sort_values(by = 'Số_điểm_Lũy_kế', ascending=False)

In [ ]:
# 
df_greater = df[df['Số_điểm_Lũy_kế']>0]
df_perc = cal_percentile(df_greater['Số_điểm_Lũy_kế'])
print('total Cus =', len(df_greater))
df_greater['perc'] = np.where((df_greater['Số_điểm_Lũy_kế'] >= 2) & (df_greater['Số_điểm_Lũy_kế'] < 40), 10,
                        np.where((df_greater['Số_điểm_Lũy_kế'] >= 40) & (df_greater['Số_điểm_Lũy_kế'] < 70), 20,
                        np.where((df_greater['Số_điểm_Lũy_kế'] >= 70) & (df_greater['Số_điểm_Lũy_kế'] < 130), 30,
                        np.where((df_greater['Số_điểm_Lũy_kế'] >= 130) & (df_greater['Số_điểm_Lũy_kế'] < 240), 40,
                        np.where((df_greater['Số_điểm_Lũy_kế'] >= 240) & (df_greater['Số_điểm_Lũy_kế'] < 630), 50,
                        np.where((df_greater['Số_điểm_Lũy_kế'] >= 630) & (df_greater['Số_điểm_Lũy_kế'] < 1200), 60,
                        np.where((df_greater['Số_điểm_Lũy_kế'] >= 1200) & (df_greater['Số_điểm_Lũy_kế'] < 2340), 70,
                        np.where((df_greater['Số_điểm_Lũy_kế'] >= 2340) & (df_greater['Số_điểm_Lũy_kế'] < 4430), 80,
                        np.where((df_greater['Số_điểm_Lũy_kế'] >= 4430) & (df_greater['Số_điểm_Lũy_kế'] < 7890), 90,
                        np.where((df_greater['Số_điểm_Lũy_kế'] >= 7890) & (df_greater['Số_điểm_Lũy_kế'] <= 1073444), 100,
                        None))))))))))
display(df_perc)
display(df_greater.groupby('perc', as_index=False).agg(
    total_cus = ('Mã thành viên','nunique'),
    total_point = ('Số_điểm_Lũy_kế', 'sum')
))

In [ ]:
df_greater[df_greater.perc >= 90].to_excel(r"C:\Users\Admin\OneDrive\FRT\6. Stored Data\AkachainMember\cus_balance_190323.xlsx")

In [ ]:
# 
df_less = df[df['Số_điểm_Lũy_kế']<0]
df_perc = cal_percentile(df_less['Số_điểm_Lũy_kế'])
print('total Cus =', len(df_less))
df_less['perc'] = np.where((df_less['Số_điểm_Lũy_kế'] >= -194546.0) & (df_less['Số_điểm_Lũy_kế'] < -7290.0), 10,
                        np.where((df_less['Số_điểm_Lũy_kế'] >= -7290.0) & (df_less['Số_điểm_Lũy_kế'] < -4948.0), 20,
                        np.where((df_less['Số_điểm_Lũy_kế'] >= -4948.0) & (df_less['Số_điểm_Lũy_kế'] < -3840.0), 30,
                        np.where((df_less['Số_điểm_Lũy_kế'] >= -3840.0) & (df_less['Số_điểm_Lũy_kế'] < -3160.0), 40,
                        np.where((df_less['Số_điểm_Lũy_kế'] >= -3160.0) & (df_less['Số_điểm_Lũy_kế'] < -2650.0), 50,
                        np.where((df_less['Số_điểm_Lũy_kế'] >= -2650.0) & (df_less['Số_điểm_Lũy_kế'] < -2090.0), 60,
                        np.where((df_less['Số_điểm_Lũy_kế'] >= -2090.0) & (df_less['Số_điểm_Lũy_kế'] < -1490.0), 70,
                        np.where((df_less['Số_điểm_Lũy_kế'] >= -1490.0) & (df_less['Số_điểm_Lũy_kế'] < -738.0), 80,
                        np.where((df_less['Số_điểm_Lũy_kế'] >= -738.0) & (df_less['Số_điểm_Lũy_kế'] < -230.0), 90,
                        np.where((df_less['Số_điểm_Lũy_kế'] >= -230.0) & (df_less['Số_điểm_Lũy_kế'] <= -10.0), 100,
                        None))))))))))
display(df_perc)
display(df_less.groupby('perc', as_index=False).agg(
    total_cus = ('Mã thành viên','nunique'),
    total_point = ('Số_điểm_Lũy_kế', 'sum')
))

In [ ]:
display(df[df['Số_điểm_Lũy_kế']<0].sort_values(by='Số_điểm_Lũy_kế'))
cal_percentile(df[df['Số_điểm_Lũy_kế']<0]['Số_điểm_Lũy_kế'])

In [ ]:
# 19/05/2023 : Export percentile again
import pandas as pd
from HelperMaster import cal_percentile
import numpy as np
df_sql = pd.read_csv(r"C:\Users\Admin\OneDrive\FRT\6. Stored Data\AkachainMember\Data_20230519_Percentile_Balance\Data_Export_SQL.csv", )
df_sql.rename(columns={'(No column name)':'total_point_sql'}, inplace=True)
df_pbi = pd.read_csv(r"C:\Users\Admin\OneDrive\FRT\6. Stored Data\AkachainMember\Data_20230519_Percentile_Balance\Data_Export_PBI.csv", )
df_pbi = df_pbi[df_pbi['IsGrandTotalRowTotal'] == False] 
df_pbi.drop(columns=('IsGrandTotalRowTotal'), inplace=True)

# Cẩn thận khi export từ sql, phải lấy type net = 0

df_merge = df_sql.merge(df_pbi, how='outer', on = 'Mã thành viên')

In [ ]:
df_pbi

In [ ]:
df_greater = df_pbi[df_pbi['Số_điểm_Lũy_kế']>0]
df_perc = cal_percentile(df_greater['Số_điểm_Lũy_kế'])
list_perc_val = df_perc['Value'].values
print('total Cus =', len(df_greater))
df_greater['perc'] = np.where((df_greater['Số_điểm_Lũy_kế'] >= list_perc_val[0]) & (df_greater['Số_điểm_Lũy_kế'] < list_perc_val[1]), 10,
                        np.where((df_greater['Số_điểm_Lũy_kế'] >= list_perc_val[1]) & (df_greater['Số_điểm_Lũy_kế'] < list_perc_val[2]), 20,
                        np.where((df_greater['Số_điểm_Lũy_kế'] >= list_perc_val[2]) & (df_greater['Số_điểm_Lũy_kế'] < list_perc_val[3]), 30,
                        np.where((df_greater['Số_điểm_Lũy_kế'] >= list_perc_val[3]) & (df_greater['Số_điểm_Lũy_kế'] < list_perc_val[4]), 40,
                        np.where((df_greater['Số_điểm_Lũy_kế'] >= list_perc_val[4]) & (df_greater['Số_điểm_Lũy_kế'] < list_perc_val[5]), 50,
                        np.where((df_greater['Số_điểm_Lũy_kế'] >= list_perc_val[5]) & (df_greater['Số_điểm_Lũy_kế'] < list_perc_val[6]), 60,
                        np.where((df_greater['Số_điểm_Lũy_kế'] >= list_perc_val[6]) & (df_greater['Số_điểm_Lũy_kế'] < list_perc_val[7]), 70,
                        np.where((df_greater['Số_điểm_Lũy_kế'] >= list_perc_val[7]) & (df_greater['Số_điểm_Lũy_kế'] < list_perc_val[8]), 80,
                        np.where((df_greater['Số_điểm_Lũy_kế'] >= list_perc_val[8]) & (df_greater['Số_điểm_Lũy_kế'] < list_perc_val[9]), 90,
                        np.where((df_greater['Số_điểm_Lũy_kế'] >= list_perc_val[9]) & (df_greater['Số_điểm_Lũy_kế'] <= list_perc_val[10]), 100,
                        None))))))))))
display(df_perc)
display(df_greater.groupby('perc', as_index=False).agg(
    total_cus = ('Mã thành viên','nunique'),
    total_point = ('Số_điểm_Lũy_kế', 'sum')
))

In [ ]:
df_greater[df_greater['perc']>=90].to_excel(r"C:\Users\Admin\OneDrive\FRT\6. Stored Data\AkachainMember\Data_20230519_Percentile_Balance\Top_20_percentile.xlsx",index=False )

In [ ]:
len(df_greater[df_greater['Số_điểm_Lũy_kế'] >=15857])

In [ ]:
# Tính thêm từ 90-100
def cal_percentile_m(df_):
    _range = np.arange(90, 101, 1)
    _percentile = np.percentile(df_, _range)
    df_return = pd.DataFrame({'Perc': _range, 'Value': _percentile})
    return df_return
df_perc_90 = cal_percentile_m(df_pbi[df_pbi['Số_điểm_Lũy_kế']>0]['Số_điểm_Lũy_kế'])
list_perc_val = df_perc_90['Value'].values
list_perc_ind = df_perc_90['Perc'].values


df_greater_90 = df_greater.copy()
df_greater_90['perc'] = np.where((df_greater_90['Số_điểm_Lũy_kế'] >= list_perc_val[0]) & (df_greater['Số_điểm_Lũy_kế'] < list_perc_val[1]), list_perc_ind[0],
                        np.where((df_greater_90['Số_điểm_Lũy_kế'] >= list_perc_val[1]) & (df_greater['Số_điểm_Lũy_kế'] < list_perc_val[2]), list_perc_ind[1],
                        np.where((df_greater_90['Số_điểm_Lũy_kế'] >= list_perc_val[2]) & (df_greater['Số_điểm_Lũy_kế'] < list_perc_val[3]), list_perc_ind[2],
                        np.where((df_greater_90['Số_điểm_Lũy_kế'] >= list_perc_val[3]) & (df_greater_90['Số_điểm_Lũy_kế'] < list_perc_val[4]), list_perc_ind[3],
                        np.where((df_greater_90['Số_điểm_Lũy_kế'] >= list_perc_val[4]) & (df_greater_90['Số_điểm_Lũy_kế'] < list_perc_val[5]), list_perc_ind[4],
                        np.where((df_greater_90['Số_điểm_Lũy_kế'] >= list_perc_val[5]) & (df_greater_90['Số_điểm_Lũy_kế'] < list_perc_val[6]), list_perc_ind[5],
                        np.where((df_greater_90['Số_điểm_Lũy_kế'] >= list_perc_val[6]) & (df_greater_90['Số_điểm_Lũy_kế'] < list_perc_val[7]), list_perc_ind[6],
                        np.where((df_greater_90['Số_điểm_Lũy_kế'] >= list_perc_val[7]) & (df_greater_90['Số_điểm_Lũy_kế'] < list_perc_val[8]), list_perc_ind[7],
                        np.where((df_greater_90['Số_điểm_Lũy_kế'] >= list_perc_val[8]) & (df_greater_90['Số_điểm_Lũy_kế'] < list_perc_val[9]), list_perc_ind[8],
                        np.where((df_greater_90['Số_điểm_Lũy_kế'] >= list_perc_val[9]) & (df_greater_90['Số_điểm_Lũy_kế'] <= list_perc_val[10]), list_perc_ind[9],
                        None))))))))))
display(df_perc_90)
display(df_greater_90.groupby('perc', as_index=False).agg(
    total_cus = ('Mã thành viên','nunique'),
    total_point = ('Số_điểm_Lũy_kế', 'sum')
))

In [ ]:
list_perc_ind

In [ ]:
df_lesser = df_pbi[df_pbi['Số_điểm_Lũy_kế']<0]
df_perc_lesser = cal_percentile(df_lesser['Số_điểm_Lũy_kế'])
list_perc_val_lesser = df_perc_lesser['Value'].values
print('total Cus =', len(df_lesser))
df_lesser['perc'] = np.where((df_lesser['Số_điểm_Lũy_kế'] >= list_perc_val_lesser[0]) & (df_lesser['Số_điểm_Lũy_kế'] < list_perc_val_lesser[1]), 10,
                        np.where((df_lesser['Số_điểm_Lũy_kế'] >= list_perc_val_lesser[1]) & (df_lesser['Số_điểm_Lũy_kế'] < list_perc_val_lesser[2]), 20,
                        np.where((df_lesser['Số_điểm_Lũy_kế'] >= list_perc_val_lesser[2]) & (df_lesser['Số_điểm_Lũy_kế'] < list_perc_val_lesser[3]), 30,
                        np.where((df_lesser['Số_điểm_Lũy_kế'] >= list_perc_val_lesser[3]) & (df_lesser['Số_điểm_Lũy_kế'] < list_perc_val_lesser[4]), 40,
                        np.where((df_lesser['Số_điểm_Lũy_kế'] >= list_perc_val_lesser[4]) & (df_lesser['Số_điểm_Lũy_kế'] < list_perc_val_lesser[5]), 50,
                        np.where((df_lesser['Số_điểm_Lũy_kế'] >= list_perc_val_lesser[5]) & (df_lesser['Số_điểm_Lũy_kế'] < list_perc_val_lesser[6]), 60,
                        np.where((df_lesser['Số_điểm_Lũy_kế'] >= list_perc_val_lesser[6]) & (df_lesser['Số_điểm_Lũy_kế'] < list_perc_val_lesser[7]), 70,
                        np.where((df_lesser['Số_điểm_Lũy_kế'] >= list_perc_val_lesser[7]) & (df_lesser['Số_điểm_Lũy_kế'] < list_perc_val_lesser[8]), 80,
                        np.where((df_lesser['Số_điểm_Lũy_kế'] >= list_perc_val_lesser[8]) & (df_lesser['Số_điểm_Lũy_kế'] < list_perc_val_lesser[9]), 90,
                        np.where((df_lesser['Số_điểm_Lũy_kế'] >= list_perc_val_lesser[9]) & (df_lesser['Số_điểm_Lũy_kế'] <= list_perc_val_lesser[10]), 100,
                        None))))))))))
display(df_perc_lesser)
display(df_lesser.groupby('perc', as_index=False).agg(
    total_cus = ('Mã thành viên','nunique'),
    total_point = ('Số_điểm_Lũy_kế', 'sum')
))

In [ ]:
df_sql.rename(columns={'(No column name)':'total_point_sql'},)

In [ ]:
df_sql

In [ ]:
# 24/03/2023
# Danh check lỗi thiếu doc
dir = r'C:\Users\Admin\OneDrive\FRT\6. Stored Data\AkachainMember\Check_Batthuong_230324'
df_thang_1 = pd.read_excel(dir + "/Thang1.xlsx", dtype={'Mã giao dịch':'object'})
df_thang_2 = pd.read_excel(dir + "/Thang2.xlsx", dtype={'Mã giao dịch':'object'})
df_thang_3 = pd.read_excel(dir + "/Thang3.xlsx", dtype={'Mã giao dịch':'object'})
df_dw = pd.read_csv( dir + '/DB.csv', dtype={'Mã giao dịch':'object'})
df_merge = pd.concat([df_thang_1, df_thang_2, df_thang_3])
# df_merge.drop(columns=['Điểm đổi quà đã sử dụng'], inplace=True)

In [ ]:
df_join =  df_merge.merge(df_dw, on= ['Lý do', 'Mã giao dịch'], how='left', suffixes=['_export', '_dw'], indicator=True)
df_join.head(3)

In [ ]:
df_join[df_join['Mã giao dịch'] == '1080100']

In [ ]:
df_join.groupby('_merge').agg(totalL = ('_merge', 'count'))

In [ ]:
df_join[df_join['_merge'] == 'left_only'].sort_values(by=['Ngày giao dịch', 'Mã giao dịch'] ).to_excel(dir + "/output_check.xlsx")

In [ ]:
# 28/03/2023: Check xem có ngày nào bị full disk nữa không
#
date_from  = '2023-03-01' 
date_to = '2023-03-20'

In [ ]:
raw_dw = pd.read_sql(f"SELECT * FROM [F_ICT_AKACHAIN_LOYALTY] WHERE [Ngày giao dịch] BETWEEN '{date_from}' AND '{date_to}' ", getEngine(SaleMart))
# raw_dw = raw_dw.drop('Điểm đổi quà đã sử dụng', inplace=True)
raw_dw.head(3)

In [ ]:
query_az = f"""SELECT  * 
            FROM [VF_CID_AKC_TRANSACTION_LOYALTY_ICT] 
            WHERE DATE_KEY 
            BETWEEN {date_from.replace('-','')} AND {date_to.replace('-','')}
            """
raw_az = pd.read_sql(query_az, azr_engine)


In [ ]:
df_az = raw_az.copy()
df_az['TRANSACTION_STATUS'] = df_az.TRANSACTION_STATUS.str.strip()
df_az['docentry_mapping'] = df_az.INVOICE_HEADER


df_dw = raw_dw.copy()


df_dw['Lý do'] = np.where(df_dw['Lý do'] == 'Burning', 'Use Point'
                , np.where(df_dw['Lý do'] == 'InvoiceReturnFull', 'ReturnFull'
                , np.where(df_dw['Lý do'] == 'InvoiceReturnPartial', 'ReturnPartial'

                , df_dw['Lý do'])))
df_dw['docentry_mapping'] = df_dw.apply(lambda x : x['Mã giao dịch'].rsplit('_',1)[1] if x['Lý do'] == 'Use Point' else x['Mã giao dịch'], axis = 1)
# df_dw[df_dw['Mã thành viên'] == '0027bd9a-db1e-42a9-b2ae-b2f53f65d70b']

# Nếu Partner Refund có use point thì là netoff
df_refund = df_dw[df_dw['Lý do'] == 'PartnerRefund'][['Mã thành viên', 'docentry_mapping','Điểm phát sinh']]
df_refund['Điểm phát sinh'] = -df_refund['Điểm phát sinh']
df_refund = df_refund.merge(df_dw[df_dw['Lý do']!= 'Invoice'], on = ['Mã thành viên', 'docentry_mapping','Điểm phát sinh'], how = 'inner')
doc_refund = df_refund.docentry_mapping.values

df_dw = df_dw[~df_dw.docentry_mapping.isin(doc_refund)]


In [ ]:
df_dw[df_dw.docentry_mapping.isin(doc_refund)]['Điểm phát sinh'].sum()

In [ ]:
len(df_dw), len(raw_dw)

In [ ]:
df_dw['Lý do'].unique()

In [ ]:
df_az.TRANSACTION_STATUS.unique()

In [ ]:
len(df_dw), len(df_az)

In [ ]:
group_az = df_az.groupby(['MEMBER_CODE_AKC', 'POINT_INFORMATION', 'docentry_mapping','TRANSACTION_STATUS'], as_index=False).agg(
    total_point_az = ('INVOICE_POINT', 'sum'),
)
group_dw = df_dw.groupby(['Mã thành viên', 'Lý do', 'docentry_mapping'], as_index=False).agg(
    total_point_dw = ('Điểm phát sinh', 'sum')
)

In [ ]:
group_dw.head(3)

In [ ]:
group_az.head(3)

In [ ]:
merge = group_az.merge(group_dw, 
                       left_on=['MEMBER_CODE_AKC', 'POINT_INFORMATION', 'docentry_mapping'],
                        right_on=['Mã thành viên', 'Lý do', 'docentry_mapping'],
                       how = 'outer',indicator=True)
merge['diff_point'] = np.where((merge._merge == 'both') & (merge.POINT_INFORMATION == 'Use Point'), merge.total_point_az.fillna(0) + merge.total_point_dw.fillna(0)
                    ,merge.total_point_az.fillna(0) - merge.total_point_dw.fillna(0)
)
merge.head(3)

In [ ]:
merge.groupby('_merge').count()

In [ ]:
# Both : Total = không diff
merge[merge._merge == 'both']['diff_point'].sum()
# merge[merge._merge == 'both'][merge.diff_point!=0]

In [ ]:
# Id gì

merge[(merge._merge == 'both') & (merge.TRANSACTION_STATUS != 'F')].head(3)
# merge.TRANSACTION_STATUS.unique()


In [ ]:
display(merge[merge._merge ==  'left_only'].groupby('POINT_INFORMATION').docentry_mapping.count())


display(merge[merge._merge ==  'right_only'].groupby('Lý do').docentry_mapping.count())


In [ ]:
merge[merge._merge ==  'left_only'].groupby('POINT_INFORMATION').docentry_mapping.min(),merge[merge._merge ==  'left_only'].groupby('POINT_INFORMATION').docentry_mapping.max()

In [ ]:
merge[merge._merge ==  'left_only'].groupby(['POINT_INFORMATION', 'TRANSACTION_STATUS']).count()

In [ ]:
merge[merge._merge ==  'right_only'].groupby('Lý do').count()

In [ ]:
merge[(merge._merge ==  'right_only') & (merge['Lý do'] == 'Use Point')]

In [ ]:
merge[(merge._merge ==  'right_only') & (merge['Lý do'] == 'Invoice')]

In [ ]:
# Hiện tại chỉ có trường hợp là
# 

In [ ]:
mem_check = 'ebd7fc47-b89a-4c56-b0f4-46af7dddd8a1'
merge[(merge.MEMBER_CODE_AKC == mem_check)|(merge['Mã thành viên'] ==mem_check)]

# Request 04/04: AB Testing

4 nhóm mỗi nhóm 2_000 khách có acumulate totalPoint > 5000


In [ ]:
# 04/04/2023
# Request: 3 nhóm mỗi nhóm 2_000 khách có acumulate totalPoint > 5000
import pandas as pd
from HelperMaster import cal_percentile
# dir = r'C:\Users\Admin\OneDrive\FRT\6. Stored Data\AkachainMember\AB_Testing_230404'
dir = r'C:\Users\Admin\OneDrive\FRT\6. Stored Data\AkachainMember\AB_Testing_230404\Round 2_230419'

# Dữ liệu point được akc verify
point_of_akc = pd.read_excel(dir + r"/Data_AKC_Verify.xlsx")

# Data point gen từ 34
df = pd.read_csv( dir + "/List_Export.csv")
df = df.rename(columns={"Mã thành viên": "id", "Monetary":"Monetary_akc"})
df = df.merge(point_of_akc[['Mã thành viên', 'Fgold', 'phone_number']], left_on= 'id', right_on='Mã thành viên', how = 'left')
df.drop(columns=['Mã thành viên', 'totalPoint'], inplace=True)
df = df.rename(columns={'Fgold':'totalPoint'})
df = df[df.Recency < 120]
df = df[df.totalPoint < 100_000]
df = df[df.totalPoint > 5_000]
df = df[df.phone_number.str[:2] =='84']

df_length_base = len(df)
print(f'Length of df {len(df):,.0f}')
display(df.head(3))

# Read data azure
df_mm = pd.read_csv(dir + "/data_mm.csv") # Data min max , trong câu SQL số 1
df_sale_agg = pd.read_csv(dir + "/data_sale_agg.csv") # Data Sale agg trong câu SQL số 2
df_sale_agg.fillna(0, inplace=True)

df = df.merge(df_mm, on='CUSTOMER_KEY', how ='inner')
print(f"Length after join mm: {len(df):,.0f}, minus: {df_length_base - len(df)}" )

df = df.merge(df_sale_agg, on='CUSTOMER_KEY', how ='inner')
print(f"Length after join sale_agg :{len(df):,.0f}, minus: {df_length_base - len(df)}" )

# # Calculate
df['two_buy_gap'] = df.MIN_MAX_BUY / df.TOTAL_BUY
df['AMT_APPLE_PERCENT'] = df.TOTAL_AMT_APPLE / df.TOTAL_AMT
df['AMT_PKNK_PERCENT'] = df.TOTAL_AMT_PKNK / df.TOTAL_AMT
df['AMT_DTDD_OTHER_PERCENT'] = df.TOTAL_AMT_DTDD_OTHER / df.TOTAL_AMT
df['AMT_PC_PERCENT'] = df.TOTAL_AMT_PC / df.TOTAL_AMT
df.columns = map(str.lower, df.columns)

df = df[df.total_amt.between(1_000_000, 500_000_000)]

display(df.head(3))

In [ ]:
cal_percentile(df['total_amt'])

In [ ]:
xx = (np.round(df['total_amt']/1_000_000,0))
from matplotlib import pyplot as plt

plt.hist(xx, bins=100, alpha=0.5, log=True)
# plt.show()


In [ ]:
# Define columns to use 
df_cal = df.copy()
att_col = [
    'totalpoint', 'recency','monetary_akc',
    'total_amt','two_buy_gap', 
    'total_buy',
    'total_amt_apple',
    'amt_apple_percent', ]

target_col = 'id'
n_cut = 4

# Using bin and qcut:
for col_ in att_col:
    print(col_)
    if col_ in (['frequency','amt_pknk_percent', 'amt_dtdd_other_percent','amt_pc_percent', 'total_buy']):
        df_cal[f'{col_}_bins'] = pd.qcut(df_cal[col_].rank(method='first'), q=n_cut, labels=[i for i in range(1, n_cut +1)])
    else:
        df_cal[f'{col_}_bins'] = pd.qcut(df_cal[col_], q=n_cut, labels=[i for i in range(1, n_cut +1)])

bins_col = [c for c in df_cal.columns if '_bins' in c]
df_cal['bin_combine'] = df_cal[bins_col].apply(
    lambda row: '_'.join(row.values.astype(str)), axis=1)

dist_of_bins = df_cal.groupby('bin_combine', as_index=False).agg(count_records = ('bin_combine', 'count'))
dist_of_bins['percent'] = dist_of_bins.count_records / dist_of_bins.count_records.sum()

n_record_treshold = 30
dist_of_bins = dist_of_bins[dist_of_bins.count_records > n_record_treshold]

print(f'Total bins: {df_cal.bin_combine.nunique()}, list bins > {n_record_treshold}: {len(dist_of_bins)}' )

#  List bins
list_bins_combine = dist_of_bins.bin_combine.unique()

In [ ]:
# Tính percent get version 2: Dynamic
N_GROUP = 4
TARGET = 2_000 * N_GROUP * 1.78
PERCENT_GET = TARGET / len(df_cal)

df_bins_label = pd.DataFrame()
for b in list_bins_combine: 
    # if b == '1_1_1_1_1_3_1_1':
        df_b = df_cal[df_cal.bin_combine == b]
        # Random DataFrame
        df_b = df_cal[df_cal.bin_combine == b]
        df_random = df_b.sample(frac=PERCENT_GET,)
        # Chia thành 3 nhóm
        group_size = df_random.shape[0] // N_GROUP
        du = df_random.shape[0] % N_GROUP
        groups = {}
        for n in range(N_GROUP):
            groups[n] = df_random.iloc[group_size * n : group_size * (n+1)].assign(flag=f'group{n}')
            
        # Gộp lại thành DataFrame mới
        df_labeled = pd.concat([groups[i] for i in range(N_GROUP)], axis=0)
        # print(len(df_labeled))
        df_bins_label = pd.concat([df_bins_label, df_labeled])

print(f"Length of result:  {len(df_bins_label):,.0f}")
print(f"Length of origin:  {len(df_cal):,.0f}")
print(f"Percent of result: {len(df_bins_label) / len(df_cal) * 100:,.3f}%")


In [ ]:
df_bins_label.groupby(['flag'], as_index=False).agg(countD = ('flag','count'))

In [ ]:
# Pair plot of distributions
import seaborn as sns
import matplotlib.pyplot as plt
pair_col = att_col.copy()
pair_col.append('flag')
g = sns.pairplot(df_bins_label[pair_col],
                 height=1.8, 
                 corner=False,
                 plot_kws={'alpha':0.3},
                 hue='flag',
                 kind='kde',
                )
# plt.show()
log_columns = ["total_amt", "total_amt_apple", 'monetary_akc', 'total_buy', ]

# for ax in g.axes.flat:
#     if ax.get_xlabel() in log_columns:
#         ax.set(xscale="log")

g.savefig(dir + "/pairplot.png")

In [ ]:
from sklearn.metrics import jaccard_score
from scipy.spatial import distance

df_g = {}
for n in range(N_GROUP):
    df_g[n] = df_bins_label[df_bins_label.flag == f'group{n}']

mins = min([len(df_g[i]) for i in range(N_GROUP) ])

print(f"Length of {N_GROUP} groups is {[len(df_g[i]) for i in range(N_GROUP) ]}")
print(f"Mins is: {mins:,.0f}")

df_final = pd.DataFrame(zip(att_col), columns= ['attributes'])
df_distance = {}
for i in range(N_GROUP):
    df_distance[i] = {}  # Initialize df_distance[i] as empty dictionary
    for y in range(i + 1, N_GROUP):
        if i != y and i < y:
            df_temp = pd.DataFrame(zip(att_col), columns= ['attributes'])
            list_temp = []
            for a_col in att_col:
                distance_iy = distance.cosine(df_g[i][:mins][a_col].values.flatten(), df_g[y][:mins][a_col].values.flatten())
                similar_rate = 1 - distance_iy
                list_temp.append(similar_rate)
            df_temp[f'similar{i}{y}'] = list_temp    
            df_final = df_final.merge(df_temp, on = 'attributes',how ='left')

display(df_final.mean())
display(df_final.style.background_gradient(cmap='YlOrRd' , vmin = 0, vmax =1))

In [ ]:
# Export result to file
df_bins_label.drop(columns= [c for c in df_bins_label.columns if '_bins' in c]).to_excel(dir + "/result.xlsx")

In [ ]:
# Lưu file vào dw để Test
import pandas as pd

dir = r'C:\Users\Admin\OneDrive\FRT\6. Stored Data\AkachainMember\AB_Testing_230404\Round 2_230419'
df_import = pd.read_excel(dir + "/result.xlsx")
df_import = df_import[['id', 'customer_key', 'flag']]
df_import.to_sql('F_ICT_AKACHAIN_LOYALTY_ABTEST_0423',getEngine(SaleMart),  if_exists='replace', index=False)

In [ ]:
# Kết quả sau khi chạy.
# Load list khách vào và so sánh
# Những thằng nào là thằng quay lại

df_back = pd.read_csv(r'C:\Users\Admin\OneDrive\FRT\6. Stored Data\AkachainMember\AB_Testing_230404\Round 2_230419\List_Final\List_Cus_Back_export_230508.csv')


In [ ]:
df_back

In [ ]:
df_result = df_cal.merge(df_back, on ='customer_key', how='inner')

# Pair plot of distributions
import seaborn as sns
import matplotlib.pyplot as plt
pair_col = att_col.copy()
pair_col.append('flag')
g = sns.pairplot(df_result[pair_col],
                 height=1.8, 
                 corner=False,
                 plot_kws={'alpha':0.3},
                 hue='flag',
                 kind='kde',
                )
# plt.show()
log_columns = ["total_amt", "total_amt_apple", 'monetary_akc', 'total_buy', ]

# for ax in g.axes.flat:
#     if ax.get_xlabel() in log_columns:
#         ax.set(xscale="log")

g.savefig(dir + "/pairplot_result.png")

# Request 07/06/2023: Check bất thường

In [ ]:
import pandas as pd
dir =  r'C:\Users\Admin\OneDrive\FRT\6. Stored Data\AkachainMember\Check_Batthuong_20230607'

df = pd.read_excel(dir + "\\AKC_TRANSACTION_20230529.xlsx")
df_tich_tieu = pd.read_csv(dir + '\\tich_tieu_sql.csv')

In [ ]:
diem = pd.read_csv(r"C:\Users\Admin\OneDrive\FRT\6. Stored Data\AkachainMember\Check_Batthuong_20230607\tich_tieu.txt", encoding = 'utf-8', sep = '\t')
diem = diem[~diem['DOCENTRY_MAPPING'].isna()]
diem = diem.astype({'DOCENTRY_MAPPING':'int'})
diem.drop(columns=['IsGrandTotalRowTotal'], inplace=True)
# diem.rename(columns={'Số_điểm_tiêu':'DIE'})
diem

In [ ]:
f_date = df[df['TRANSACTION_TYPE'] == 'Sales Invoice'].groupby('INVOICE_HEADER', as_index=False).agg(MIN_DATE_BUY = ("DATE_KEY", 'min'))
f_date['MIN_DATE_BUY'] = pd.to_datetime(f_date['MIN_DATE_BUY'], format = '%Y%m%d')
f_date

In [ ]:
group_col = ['MÃ THÀNH VIÊN', 'CUSTOMER_NAME', 'CUSTOMER_PHONE', 'CUSTOMER_KEY','TRANSACTION_TYPE','INVOICE_HEADER']
df_g = df.groupby(group_col, as_index=False).agg(
    totalAmount = ('LIST_AMT_AF_VAT', 'sum'),
    totalProduct = ('PRODUCT_NAME', 'nunique'),
    totalCategory = ('PRODUCT_CATEGORY_NAME', 'nunique'),
    totalCategory_level_2 = ('PRODUCT_GROUP_NAME', 'nunique'),
)

df_product_name = df.groupby(group_col)['PRODUCT_NAME'].apply(list).reset_index(name='PRODUCT_NAME')
df_g = df_g.merge(df_product_name, on=group_col)

df_g['PRODUCT_NAME'] = df_g['PRODUCT_NAME'].apply(lambda x: '\n'.join(x) if isinstance(x, list) else x)


# df_g.to_excel(dir + "/result.xlsx")

In [ ]:
df_final = df_g.merge(diem, left_on= 'INVOICE_HEADER', right_on='DOCENTRY_MAPPING', how='left').drop(columns = ['DOCENTRY_MAPPING'])
df_final = df_final.merge(f_date, on ='INVOICE_HEADER', how = 'left')
# df_final = df_final[df_final['TRANSACTION_TYPE'] == 'Sales Invoice']
# df_final['Số_điểm_tiêu'] = np.where(df_final['TRANSACTION_TYPE'] != 'Sales Invoice', 0 , df_final['Số_điểm_tiêu'])
# df_final['Số_điểm_tích_lũy'] = np.where(df_final['TRANSACTION_TYPE'] != 'Sales Invoice', 0 , df_final['Số_điểm_tích_lũy'])
# df_final
# df_final = df_final.sort_values(by = ['MÃ THÀNH VIÊN', 'MIN_DATE_BUY'], ascending = [True,True]) 
# df_final.to_excel(dir + "/result.xlsx", index = False)

In [ ]:
df_final['Số_điểm_tiêu'].sum(), df_final['Số_điểm_tích_lũy'].sum()

In [ ]:
df_tich_tieu = pd.read_csv(dir + '\\tich_tieu_sql.csv')
df_tich_tieu['type'] = np.where(df_tich_tieu['Lý do'].str.contains('Invoice'), 'Tich',
                                np.where

df_tich_tieu[df_tich_tieu['type'] == 1]['Lý do'].unique()